In [3]:
import sys
!{sys.executable} -m pip install py-algorand-sdk

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 KB 457.3 kB/s eta 0:00:001m448.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 KB 2.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 KB 2.3 MB/s eta 0:00:00 MB/s eta 0:00:01


In [4]:
from algosdk import account, mnemonic
from algosdk import transaction

# Create Account

## Account 1

In [47]:
private_key, address = account.generate_account()
print(f"address: {address}")
print(f"private key: {private_key}")
print(f"mnemonic: {mnemonic.from_private_key(private_key)}")

address: 2L64GNWDEEH5G5RN4G6OUADOJZJB3YX6WY2ROBGKTYGIJ3RCB7URUPNJ7I
private key: jTjqQcLVN3mdyOfHarCPP6F46XItzvEHGTj3wVXnqvjS/cM2wyEP03Yt4bzqAG5OUh3i/rY1FwTKngyE7iIP6Q==
mnemonic: balance pottery can found hurry furnace eager treat flight script sick become vanish ripple high soda wreck goat orchard buyer process right shed abstract record


## Account 2

In [15]:
private_key2, address2 = account.generate_account()
print(f"address2: {address2}")
print(f"private key2: {private_key2}")
print(f"mnemonic2: {mnemonic.from_private_key(private_key2)}")

address2: ZP2QA3HVLBUEJKRW6N6WETSFQIFPA6IQUULH4MVUEB6ALAS43BTCTHK6YE
private key2: Bm8UO77plDlFbh9z3c9FKz93csoo2TyCyIa/91JbPO3L9QBs9VhoRKo2831iTkWCCvB5EKUWfjK0IHwFglzYZg==
mnemonic2: this faculty budget orbit clap example impulse buyer purpose disorder pepper verify unveil orient edge summer elegant capital sell know planet bitter hazard abstract cargo


In [1]:
from utils import get_accounts, get_algod_client

# Connect to Client

In [10]:
from algosdk.v2client import algod

algod_address = "http://localhost:4001"
algod_token = "a" * 64
algod_client = algod.AlgodClient(algod_token, algod_address)

# Or, if necessary, pass alternate headers

# Create a new client with an alternate api key header
special_algod_client = algod.AlgodClient(
    "", algod_address, headers={"X-API-Key": algod_token}
)

In [10]:
algod_client = get_algod_client()
accts = get_accounts()

acct1 = accts.pop()
private_key, address = acct1.private_key, acct1.address
address

'3O3IITONCTC24X77YG5AHR4NGUT7VFYTMJIYYIAL7CKOS6S4FVQM6MWFDA'

In [12]:
acct2 = accts.pop()
address2 = acct2.address
address2

'A2243ENLO6PN4EAX3SM3K6KQ2OM66QUAP2Y4FSRY72TDVRQT6HWFK3X2NM'

# Check Balance

In [9]:
from typing import Dict, Any

account_info: Dict[str, Any] = algod_client.account_info(address2)
print(f"Account balance: {account_info.get('amount')} microAlgos")

Account balance: 4000000001000000 microAlgos


# Make First Transaction

In [17]:
params = algod_client.suggested_params()
unsigned_txn = transaction.PaymentTxn(
    sender=address,
    sp=params,
    receiver=address2,
    amt=1000000,
    note=b"Hello World",
)
unsigned_txn

# Sign Transaction

In [18]:
# sign the transaction
signed_txn = unsigned_txn.sign(private_key)
signed_txn

# Submit the Transaction

In [19]:
import json
from base64 import b64decode
# submit the transaction and get back a transaction id
txid = algod_client.send_transaction(signed_txn)
print("Successfully submitted transaction with txID: {}".format(txid))

# wait for confirmation
txn_result = transaction.wait_for_confirmation(algod_client, txid, 4)

print(f"Transaction information: {json.dumps(txn_result, indent=4)}")
print(f"Decoded note: {b64decode(txn_result['txn']['txn']['note'])}")

Successfully submitted transaction with txID: JL6MK2LQWU5XP4AMBMGXHZ46TAZZVQNOIP2UUVKNO5CUCHCB5CZA
Transaction information: {
    "confirmed-round": 3,
    "pool-error": "",
    "txn": {
        "sig": "AJ5M5AO+ep6TIei1bTKM+6pIo0oDkqbrac+7l3ifPNtAiT6DZGPnBgLMWNglTGwwVg/gwpFeKndyNUdUtZVMAQ==",
        "txn": {
            "amt": 1000000,
            "fee": 1000,
            "fv": 2,
            "gen": "sandnet-v1",
            "gh": "EuOLWbil6m3MlKJH5C2JAZfd6KQBrSto1PsQFbplc2M=",
            "lv": 1002,
            "note": "SGVsbG8gV29ybGQ=",
            "rcv": "A2243ENLO6PN4EAX3SM3K6KQ2OM66QUAP2Y4FSRY72TDVRQT6HWFK3X2NM",
            "snd": "3O3IITONCTC24X77YG5AHR4NGUT7VFYTMJIYYIAL7CKOS6S4FVQM6MWFDA",
            "type": "pay"
        }
    }
}
Decoded note: b'Hello World'
